In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

In [4]:
from keras.layers import Input, Lambda, Dense, Flatten

In [5]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'

In [7]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [8]:
for layer in vgg.layers:
    layer.trainable = False

In [9]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
522/522 [==============================] - 228s 429ms/step - loss: 0.2216 - accuracy: 0.9266 - val_loss: 0.3181 - val_accuracy: 0.8974
Epoch 2/5
522/522 [==============================] - 127s 243ms/step - loss: 0.1366 - accuracy: 0.9553 - val_loss: 0.6621 - val_accuracy: 0.8413
Epoch 3/5
522/522 [==============================] - 126s 242ms/step - loss: 0.1261 - accuracy: 0.9569 - val_loss: 1.2943 - val_accuracy: 0.7885
Epoch 4/5
522/522 [==============================] - 128s 246ms/step - loss: 0.1535 - accuracy: 0.9565 - val_loss: 0.8903 - val_accuracy: 0.8830
Epoch 5/5
522/522 [==============================] - 130s 248ms/step - loss: 0.1109 - accuracy: 0.9678 - val_loss: 0.4102 - val_accuracy: 0.9054


In [15]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')

In [16]:
from keras.models import load_model

In [22]:
from tensorflow.keras.preprocessing import image

In [23]:
from keras.applications.vgg16 import preprocess_input

In [24]:
import numpy as np

In [25]:
model=load_model('chest_xray.h5')

In [36]:
img=image.load_img(r'/content/drive/MyDrive/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg' ,target_size=(224,224))

In [38]:
x=image.img_to_array(img)

In [39]:
x=np.expand_dims(x, axis=0)

In [40]:
img_data=preprocess_input(x)

In [41]:
classes=model.predict(img_data)

1/1 [==============================] - 0s 24ms/step


In [42]:
result=int(classes[0][0])

In [43]:
if result==0:
    print("Person is Affected By PNEUMONIA")
else:
    print("Result is Normal")

Person is Affected By PNEUMONIA
